In [3]:
import pandas as pd
import numpy as num
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import matplotlib
import seaborn as sns
import statistics as st
from scipy import integrate as intg
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from bioinfokit.analys import stat
import os
from statsmodels.graphics.factorplots import interaction_plot
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [4]:
root = 'Chapter1_three_bin'
growingseasonsdf = pd.read_csv("growing_seasons_all.csv")
dailydf = pd.read_csv("AMF_US-MOz_FLUXNET_SUBSET_DD_2004-2019_3-5.csv")
halfhourlydf = pd.read_csv("AMF_US-MOz_FLUXNET_SUBSET_HH_2004-2019_3-5.csv")
halfhourlydf = halfhourlydf.replace(-9999, num.NaN)

#inner joining growingseasondf and dailydf to isolate growing seasons
growingseasonsdailydf = pd.merge(growingseasonsdf, dailydf, on='TIMESTAMP')

#empty column for did it rain? your name and stress lvl flags
growingseasonsdailydf['PRECIP_YN'] = num.nan
growingseasonsdailydf['STRESS_LVL'] = num.nan

#function for first day of growing seasons
def first_day_precip(df: pd.DataFrame, ind):
    if df.at[ind, 'P_F']>=5:
        df.at[ind, 'PRECIP_YN']= 'Wet'
    else:
        df.at[ind, 'PRECIP_YN']= 'Dry'

for ind in growingseasonsdailydf.index:
    #precip condition flagging
    #first day of entire growingseasondailydf
    if ind == 0:
        first_day_precip(growingseasonsdailydf, ind)
    else:
        counter=growingseasonsdailydf.at[ind-1, 'index']
        prevrain=growingseasonsdailydf.at[ind-1, 'P_F']
        #check to see if in same year
        if growingseasonsdailydf.at[ind, 'index']>counter:
            #if same year, mark precip_yn
            if growingseasonsdailydf.at[ind, 'P_F']>=5 and prevrain>=5:
                growingseasonsdailydf.at[ind, 'PRECIP_YN']='wWet'
            elif growingseasonsdailydf.at[ind, 'P_F']>=5 and prevrain<5:
                growingseasonsdailydf.at[ind, 'PRECIP_YN']='dWet'
            elif growingseasonsdailydf.at[ind, 'P_F']<5 and prevrain>=5:
                growingseasonsdailydf.at[ind, 'PRECIP_YN']='wDry'
            elif growingseasonsdailydf.at[ind, 'P_F']<5 and prevrain<5:
                growingseasonsdailydf.at[ind, 'PRECIP_YN']='dDry'
            else:
                print(ind)
        #if not in same year, don't count day before
        else:
            first_day_precip(growingseasonsdailydf, ind)

    #stress level flagging
    if growingseasonsdailydf.at[ind, ' PLWP']<0 and growingseasonsdailydf.at[ind, ' PLWP']>= -0.5:
        growingseasonsdailydf.at[ind, 'STRESS_LVL']='Low'
    elif growingseasonsdailydf.at[ind, ' PLWP']< -0.5 and growingseasonsdailydf.at[ind, ' PLWP']>= -1:
        growingseasonsdailydf.at[ind, 'STRESS_LVL']='Medium'
    elif growingseasonsdailydf.at[ind, ' PLWP']< -1:
        growingseasonsdailydf.at[ind, 'STRESS_LVL']='High'
    else:
        print(ind)

In [5]:
#create empty lists for each bin i.e. wWet AND mediumLow
dDryLow = []
dDryMedium = []
dDryHigh = []

wDryLow = []
wDryMedium = []
wDryHigh = []

dWetLow = []
dWetMedium =[]
dWetHigh = []

wWetLow =[]
wWetMedium = []
wWetHigh = []

#sort timestamps into respective bins
for ind in growingseasonsdailydf.index:
    if growingseasonsdailydf.at[ind, 'PRECIP_YN']=='dDry' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='Low':
        dDryLow.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))
    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='dDry' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='Medium':
        dDryMedium.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))
    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='dDry' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='High':
        dDryHigh.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))

    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='wDry' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='Low':
        wDryLow.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))
    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='wDry' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='Medium':
        wDryMedium.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))
    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='wDry' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='High':
        wDryHigh.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))

    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='dWet' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='Low':
        dWetLow.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))
    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='dWet' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='Medium':
        dWetMedium.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))
    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='dWet' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='High':
        dWetHigh.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))

    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='wWet' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='Low':
        wWetLow.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))
    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='wWet' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='Medium':
        wWetMedium.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))
    elif growingseasonsdailydf.at[ind, 'PRECIP_YN']=='wWet' and growingseasonsdailydf.at[ind, 'STRESS_LVL']=='High':
        wWetHigh.append(str(growingseasonsdailydf.at[ind, 'TIMESTAMP']))
    else:
        #going to default to this on the first days of the growing seasons where it only has wet or dry
        continue

#create new column in halfhourly df to inner join
halfhourlydf = halfhourlydf.astype({'TIMESTAMP_START':'string'})
halfhourlydf['TIMESTAMP']=halfhourlydf['TIMESTAMP_START'].str[:8]

In [6]:
def calc_eWUE(df: pd.DataFrame):
    df['lambda'] = (2.501 - (df['TA_F'] * 0.002361))*1000000
    df['ET'] = df['LE_F_MDS'] / df['lambda']
    df['GPP_NT_VUT_50'] = df['GPP_NT_VUT_50'] * 0.001
    df['eWUE'] = df['GPP_NT_VUT_50'] / (df['ET']) / 1000 * 18.02
def calc_LUE(df: pd.DataFrame):
    # df['PAR'] = df['SW_IN_F'] / 2 / 235000
    df['PAR'] = df['PPFD_IN'] - df['PPFD_OUT']
    df['LUE'] = df['GPP_NT_VUT_50'] / df['PAR'] * 1000

#if nighttime, set GPP to nan <--- we only want to calculate eWUE and LUE for daytime values
halfhourlydf.loc[halfhourlydf['SW_IN_F'] <=100, 'GPP_NT_VUT_50'] = num.nan
#create columns to hold only daytime values for LE and H for future EF calculations
halfhourlydf['LE_DAYTIME']=halfhourlydf['LE_F_MDS']
halfhourlydf['H_DAYTIME']=halfhourlydf['H_F_MDS']
halfhourlydf.loc[halfhourlydf['SW_IN_F'] <=5, 'LE_DAYTIME'] = num.nan
halfhourlydf.loc[halfhourlydf['SW_IN_F'] <=5, 'H_DAYTIME'] = num.nan
#if LE is = 0 then set to nan, LE shouldn't be 0 and is only zero because it is gap filled, so this avoids that 
halfhourlydf.loc[halfhourlydf['LE_F_MDS']==0, 'LE_F_MDS'] = num.nan
halfhourlydf.loc[halfhourlydf['H_F_MDS']==0, 'H_F_MDS'] = num.nan

calc_eWUE(halfhourlydf)
calc_LUE(halfhourlydf)
#reasonable range filters for eWUE and LUE
halfhourlydf.loc[halfhourlydf['eWUE']>=40, 'eWUE'] = num.nan
halfhourlydf.loc[halfhourlydf['eWUE']<= 0, 'eWUE'] = num.nan
# halfhourlydf.loc[halfhourlydf['LUE']> 0.1, 'LUE'] = num.nan
halfhourlydf.loc[halfhourlydf['LUE']< 0, 'LUE'] = num.nan

In [11]:
#empty parent lists to put stats lists into
eflistforanova = []
nee_min = []
nee_max = []
nee_int = []
le_max = []
le_int = []
h_max = []
h_int = []
netrad_max = []
netrad_int = []
centroid_ = []

#create empty dfs to put in stats
needf = pd.DataFrame(columns=['BIN', 'MIN', 'MIN_STD', 'MAX', 'MAX_STD','INT', 'INT_STD'])
ledf = pd.DataFrame(columns=['BIN', 'MIN', 'MIN_STD', 'MAX', 'MAX_STD', 'INT', 'INT_STD'])
hdf = pd.DataFrame(columns=['BIN', 'MIN', 'MIN_STD', 'MAX', 'MAX_STD', 'INT', 'INT_STD'])
netraddf = pd.DataFrame(columns=['BIN', 'MIN', 'MIN_STD', 'MAX', 'MAX_STD', 'INT', 'INT_STD'])
efdf = pd.DataFrame(columns=['BIN', 'MEAN', 'STD_DEV'])
centroiddf = pd.DataFrame(columns=['BIN', 'Time', 'SE'])

In [24]:
#take variables and put into matrix so that columns are days and rows are half hours
def variable_matrix(df: pd.DataFrame, variable: str, n):
    varlist = df[variable].values.tolist()
    varlist = num.reshape(varlist, (n, 48))
    varlist = num.transpose(varlist)
    return varlist

#weighted centroid function, taken from Nelson et al. 2018
def diurnal_centroid(flux, bin, unitesperday=48):
    days, UPD = flux.reshape(-1,unitesperday).shape
    hours = num.tile(num.arange(UPD),days).reshape(days,UPD)
    C=num.sum(hours*flux.reshape(-1, 48), axis=1)/num.sum(flux.reshape(-1,48), axis=1)
    C = C*(24/unitesperday)
    C, SE = mean_stddev(C, days)
    cenlist = [bin, C, SE]
    centroiddf.loc[len(centroiddf.index)] = cenlist

#spitting out SE not SD anymore
def mean_stddev(list, n):
    mean = num.nanmean(list)
    std_dev = num.nanstd(list)
    std_err = std_dev / num.sqrt(n)
    return mean, std_err

def variable_stats(list, n, bin: str, var:str):
    #create lists with stats from every day
    minlist = []
    maxlist = []
    intlist = []
    #centroidlist = []
    # for each column, aka, for each day within bin
    for i in range(n):
        col = list[:, i]
        daytime_col = list[12:36, i]
        if num.isnan(daytime_col).all():
            continue
        else:
            mini = num.nanmin(col)
            maxi = num.nanmax(col)
            minlist.append(mini)
            maxlist.append(maxi)
            integral = [x*1800 for x in col]
            intlist.append(sum(integral))
        # if var == 'nee':
        #     #centroid function
        #     hours = num.arange(0, 48)
        #     C=num.sum(hours*col)/num.sum(col)
        #     C = C*(24/48)
        #     centroidlist.append(C)
   
    #put stats lists into parent lists to save for later for anova
    if var == 'nee':
        # centroid_.append(centroidlist)
        # cen_mean, cen_std = mean_stddev(centroidlist, n)
        # cenlist = [bin, cen_mean, cen_std]
        # centroiddf.loc[len(centroiddf.index)] = cenlist

        nee_min.append(minlist)
        intlist = [x/1000 for x in intlist]
        nee_int.append(intlist)
        nee_max.append(maxlist)
    elif var == 'le':
        le_max.append(maxlist)
        intlist = [x/1000000 for x in intlist]
        le_int.append(intlist)
    elif var == 'h':
        h_max.append(maxlist)
        intlist = [x/1000000 for x in intlist]
        h_int.append(intlist)
    elif var == 'netrad':
        netrad_max.append(maxlist)
        intlist = [x/1000000 for x in intlist]
        netrad_int.append(intlist)
    else:
        print('invaild var: str input in variable_stats')
    
    #find average of those lists, all SDs are now SEs
    min_mean, min_std = mean_stddev(minlist, n)
    max_mean, max_std = mean_stddev(maxlist, n)
    int_mean, int_std = mean_stddev(intlist, n)
    #return list with all those stats
    statslist = [bin, min_mean, min_std, max_mean, max_std, int_mean, int_std] 
    return statslist

def statistic_calcs(df: pd.DataFrame, n, bin: str):
    # df is ddl, ddm, ddh
    # n is number of days 
    # bin is a string 'ddl', 'ddm', 'ddh'
    neelist = variable_matrix(df, 'NEE_VUT_REF', n)
    neelist = variable_stats(neelist, n, bin, 'nee')
    needf.loc[len(needf.index)] = neelist

    lelist = variable_matrix(df, 'LE_F_MDS', n)
    lelist = variable_stats(lelist, n, bin, 'le')
    ledf.loc[len(ledf.index)] = lelist

    hlist = variable_matrix(df, 'H_F_MDS', n)
    hlist = variable_stats(hlist, n, bin, 'h')
    hdf.loc[len(hdf.index)]=hlist

    netradlist = variable_matrix(df, 'NETRAD', n)
    netradlist = variable_stats(netradlist, n, bin, 'netrad')
    netraddf.loc[len(netraddf.index)]=netradlist

def efints(criteria:pd.DataFrame, bin:str, n):
    #ef calculation with LE an H daytime integrals
    LElist = variable_matrix(criteria, 'LE_DAYTIME', n)
    Hlist = variable_matrix(criteria, 'H_DAYTIME', n)
    LEintlist = []
    Hintlist = []
    for i in range(n):
        daytimeH = Hlist[:, i]
        daytimeH = [x for x in daytimeH if ~num.isnan(x) == True]
        daytimeLE = LElist[:, i]
        daytimeLE = [x for x in daytimeLE if ~num.isnan(x) == True]

        # xdata = num.sum(~num.isnan(Hlist[:,i]))
        # xdataH = num.arange(0, xdata)
        # xdataLE = num.sum(~num.isnan(LElist[:,i]))
        # xdataLE = num.arange(0, xdataLE)
        # LEint = intg.trapz(daytimeLE, xdataLE)
        # Hint = intg.trapz(daytimeH, xdataH)

        LEint = [x*1800 for x in daytimeLE]
        LEint = sum(LEint)
        Hint = [x*1800 for x in daytimeH]
        Hint = sum(Hint)
        LEint = round(LEint, 4)
        Hint = round(Hint, 4)
        LEintlist.append(LEint)
        Hintlist.append(Hint)
    # keeping ints stored in case need for something else
    efintsdf = pd.DataFrame({'LEint': LEintlist, 'Hint': Hintlist})
    #calculate EF
    efintsdf['EF'] = efintsdf['LEint'] / (efintsdf['LEint'] + efintsdf['Hint'])
    eflist = efintsdf['EF'].values.tolist()
    eflistforanova.append(eflist)
    ef_mean, ef_std = mean_stddev(eflist, n)
    eflist = [bin, ef_mean, ef_std]
    efdf.loc[len(efdf.index)]= eflist

In [11]:
def calc_taunc(df: pd.DataFrame):
    df['TA_INSTUNC']=None
    df['RH_INSTUNC']=None
    df['PA_INSTUNC']=None
    for ind in df.index:
        if df.loc[ind, 'TA_F'] > 20:
            df.loc[ind, 'TA_INSTUNC'] = 0.055 + 0.0057 * df.loc[ind, 'TA_F']
        else:
            df.loc[ind, 'TA_INSTUNC'] = 0.226 - 0.0028 * df.loc[ind, 'TA_F']
        
        if df.loc[ind, 'TA_F'] < -40:
            df.loc[ind, 'RH_INSTUNC'] = 1.4 + 0.032*df.loc[ind, 'RH'] 
        elif df.loc[ind, 'TA_F'] < -20:
            df.loc[ind, 'RH_INSTUNC'] = 1.2 + 0.012*df.loc[ind, 'RH']
        elif df.loc[ind, 'TA_F'] < 40:
            df.loc[ind, 'RH_INSTUNC'] = 1 + 0.008*df.loc[ind, 'RH']
        elif df.loc[ind, 'TA_F'] < 60:
            df.loc[ind, 'RH_INSTUNC'] = 1.2 + 0.012*df.loc[ind, 'RH']

        if df.loc[ind, 'TA_F'] >= 15 and df.loc[ind, 'TA_F'] <= 25:
            if df.loc[ind, 'RH'] < 90:
                df.loc[ind, 'RH_INSTUNC'] = 1
            else: df.loc[ind, 'RH_INSTUNC'] = 1.7
        
        if 15 <= df.loc[ind, 'TA_F'] <= 25:
            df.loc[ind, 'PA_INSTUNC'] = 0.5
        elif 0 <= df.loc[ind, 'TA_F'] <= 40:
            df.loc[ind, 'PA_INSTUNC'] = 1
        elif -20 <= df.loc[ind, 'TA_F'] <= 50:
            df.loc[ind, 'PA_INSTUNC'] = 1.5
        elif -40 <= df.loc[ind, 'TA_F'] <= 60:
            df.loc[ind, 'PA_INSTUNC'] = 2

    df['TA_INSTUNC'] = df['TA_INSTUNC'].astype(float)
    df['RH_INSTUNC'] = df['RH_INSTUNC'].astype(float)
    df['VPD_INSTUNC'] = df['VPD_F']*(num.sqrt(((df['TA_INSTUNC']/df['TA_F'])**2)+((df['RH_INSTUNC']/df['RH'])**2)))
    df['PA_INSTUNC'] = df['PA_INSTUNC'].astype(float)
    ta = ((df['TA_INSTUNC'] / df['TA_F'])**2)*1000000
    le = (df['LE_RANDUNC'] / df['LE_F_MDS'])**2
    gpp = ((df['GPP_NT_VUT_REF'] / df['GPP_NT_VUT_50'])**2)*0.001
    df['eWUE_INSTUNC'] = df['eWUE'] * (num.sqrt(ta+le+gpp)*18.01/1000)
    gpplue = ((df['GPP_NT_VUT_REF'] / df['GPP_NT_VUT_50'])**2)
    par = (df['PAR_UNC'] / df['PAR'])**2
    df['LUE_INSTUNC'] = df['LUE'] * (num.sqrt(gpplue + par)*1000)
    df['PA_F'] = df['PA_F']*10

    return df

def net_std_calc(df, row_index, window_size, rg_thres=50, ta_thres=2.5, vpd_thres=5.0):
    startdate = df['TIMESTAMP'][row_index] - pd.DateOffset(days=window_size)
    enddate = df['TIMESTAMP'][row_index] + pd.DateOffset(days=window_size)

    mask = (
        (df['TIMESTAMP'] >= startdate) & (df['TIMESTAMP'] <= enddate) &
        (abs(df['NETRAD'] - df['NETRAD'][row_index]) <= rg_thres) &
        (abs(df['TA_F'] - df['TA_F'][row_index]) <= ta_thres) &
        (abs(df['VPD_F'] - df['VPD_F'][row_index]) <= vpd_thres)
    )
    selected_data = df[mask]
    if not selected_data.empty:
        std_devnet = selected_data['NETRAD'].std()
        std_devpar = selected_data['PAR'].std()
    else:
        std_devnet = num.nan
        std_devpar = num.nan
    return std_devnet, std_devpar    

def netraduncs(df: pd.DataFrame):
    uncdf = df[['TIMESTAMP','NETRAD', 'TA_F', 'VPD_F', 'PAR']].copy()
    uncdf['TIMESTAMP'] = pd.to_datetime(uncdf['TIMESTAMP'], format= '%Y%m%d')

    window_size = 7
    std_dev_netvalues = []
    std_dev_parvalues = []

    for i in range(len(uncdf)):
        std_devnet, std_devpar = net_std_calc(uncdf, i, window_size)
        std_dev_netvalues.append(std_devnet)
        std_dev_parvalues.append(std_devpar)
    
    return std_dev_netvalues, std_dev_parvalues

In [25]:
def createHalfHourAvgDFs(criteria, bin:str):
    #convert to df for inner join then join
    N = len(criteria)
    # print(bin, N)
    criteria=pd.DataFrame(criteria, columns=['TIMESTAMP'])
    criteria=pd.merge(criteria, halfhourlydf, on='TIMESTAMP')

    #creating uncertainties for NETRAD and PAR... computationally expensive, don't need to run every time
    # criteria['NETRAD_UNC'], criteria['PAR_UNC'] = netraduncs(criteria)
    # criteria = calc_taunc(criteria)

    #saving to csv file
    # csv = criteria.to_csv(root + '/' + bin + ".csv")

    #calculate EF integrals 
    #efints(criteria, bin, N)
    #statistics before averaging all half hours for graphing
    statistic_calcs(criteria, N, bin)
    
    #average timesteps
    criteria['HalfHour']=criteria['TIMESTAMP_START'].str[8:]
    criteria=criteria.groupby(['HalfHour']).mean(numeric_only = True)
    criteria=criteria.reset_index()

    # csv = criteria.to_csv(root + '/' + bin + "48.csv")

    return criteria

In [26]:
#run all the functions above to create stats dfs and dfs with average daily cycle
diurnal_centroid(dDryLow['NEE_VUT_REF'], 'ddl', unitesperday=48)

# dDryLow= createHalfHourAvgDFs(dDryLow, 'ddl')
# dDryMedium= createHalfHourAvgDFs(dDryMedium, 'ddm')
# dDryHigh= createHalfHourAvgDFs(dDryHigh, 'ddh')

# dWetLow= createHalfHourAvgDFs(dWetLow, 'dwl')
# dWetMedium= createHalfHourAvgDFs(dWetMedium, 'dwm')
# dWetHigh= createHalfHourAvgDFs(dWetHigh, 'dwh')

# wWetLow= createHalfHourAvgDFs(wWetLow, 'wwl')
# wWetMedium= createHalfHourAvgDFs(wWetMedium, 'wwm')
# wWetHigh= createHalfHourAvgDFs(wWetHigh, 'wwh')

# wDryLow= createHalfHourAvgDFs(wDryLow, 'wdl')
# wDryMedium= createHalfHourAvgDFs(wDryMedium, 'wdm')
# wDryHigh= createHalfHourAvgDFs(wDryHigh, 'wdh')

AttributeError: 'Series' object has no attribute 'reshape'

In [ ]:
#anova things
maindfnamelist = ['dDryLow', 'dDryMedium', 'dDryHigh', 'dWetLow', 'dWetMedium', 'dWetHigh', 'wWetLow', 'wWetMedium', 'wWetHigh', 'wDryLow', 'wDryMedium', 'wDryHigh']

def anova_tukey(listoflists, dfnamelist):
    #listoflists is array with one variable and one stat, example: NEE_INT
    #next line has rows = all nee ints for one bin, columns = number of days in each bin
    df=pd.DataFrame(listoflists)
    #switch to rows = number of days in each bin, columns = bin
    df = num.transpose(df)
    #add labels for anova
    df = df.set_axis(dfnamelist, axis=1, copy=False)
    data = [df[col].dropna() for col in df]
    f_val, p_val = stats.f_oneway(*data)
    df_melt = pd.melt(df.reset_index(), id_vars =['index'], value_vars=dfnamelist)
    df_melt.columns = ['index', 'treatments', 'value']
    res = stat()
    #future warning error is because xfac_var has nans
    res.tukey_hsd(df=df_melt, res_var='value', xfac_var='treatments', anova_model='value ~C(treatments)')
    return f_val, p_val, res.tukey_summary 

def anova(listoflists, dfnamelist):
    df=pd.DataFrame(listoflists)
    df=num.transpose(df)
    df=df.set_axis(dfnamelist, axis=1, copy=False)
    df_melt=pd.melt(df.reset_index(), id_vars=['index'], value_vars=dfnamelist)
    df_melt.columns=['index', 'treatments', 'value']
    df_melt['pc']=df_melt['treatments'].str[:4]
    df_melt['stress']=df_melt['treatments'].str[4:]
    df_melt=df_melt.drop(['index', 'treatments'], axis=1)
    df_melt=df_melt.dropna()
    df_melt=df_melt.reset_index()

    formula = 'value ~ C(pc) * C(stress)'
    model=ols(formula, data=df_melt).fit()
    anova_table=sm.stats.anova_lm(model, typ=3)
    print(anova_table)
    print()
    print(model.summary())


    pc_levels = df_melt['pc'].unique()
    stress_levels = df_melt['stress'].unique()

    lsmeans = []
    for pc_level in pc_levels:
        for stress_level in stress_levels:
            term = f'C(pc):C(stress)'
            lsmean = model.get_robustcov_results().t_test([term])
            lsmeans.append((f'PC{pc_level}_Stress{stress_level}', lsmean))

    # Print the least squares means
    print("\nLeast Squares Means:")
    for lsmean in lsmeans:
        print(f"{lsmean[0]}: {lsmean[1].effect[0]:.4f}")

    # Calculate Least Squares Means (LSMeans)
    # tukey_results = pairwise_tukeyhsd(df_melt['value'], df_melt['pc'] + df_melt['stress'])
    # lsmeans = pd.DataFrame(tukey_results.meandiffs, columns=['LSMeans'])
    # lsmeans['CI_lower'] = tukey_results.confint[:, 0]
    # lsmeans['CI_upper'] = tukey_results.confint[:, 1]

    # print(lsmeans)
    # tukey_results = pairwise_tukeyhsd(df_melt['value'], df_melt['pc'] + df_melt['stress'])
    # #print(tukey_results)
    # ls_means = tukey_results.meandiffs + df_melt['value'].mean()

    # print(ls_means)
    # fig=interaction_plot(x=df_melt['pc'], trace=df_melt['stress'], response=df_melt['value'], colors=['#4c061d','#d17a22', '#b4c292'])
    # plt.show()   

#currently outputting the differences of all the combinations of groups of their least square means, but i want ONLY the least square means
anova(h_int, maindfnamelist)
# fh, ph, th = anova_tukey(h_int, maindfnamelist)
# print(fh)
# print(ph)
# print(th)